## Amaon web scraper 

# with :
#### beautifulsoup
#### selenium


In [3]:
import csv
from bs4 import BeautifulSoup

In [8]:
# firefox and chrom
from selenium import webdriver 

# startup the web driver 

In [16]:
driver = webdriver.Firefox()
driver = webdriver.Chrome()

In [17]:
url = 'https://www.amazon.com'
driver.get(url)

In [18]:
def get_url(search_term):
    """generate a url from a search term"""
    template = 'https://www.amazon.com/s?k={}'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [19]:
url =get_url('ultrawide mpnitor')
print(url)

https://www.amazon.com/s?k=ultrawide+mpnitor


In [20]:
driver.get(url)

## extract the collection

In [22]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [23]:
results = soup.find_all('div',{'data-component-type':'s-search-result'})

In [24]:
len(results)

22

## prototype the record 

In [40]:
item = results[0]
atag = item.h2.a
description = atag.text.strip()
url = 'https://www.amazon.com'+atag.get('href')

In [41]:
price_parent = item.find('span','a-price')
price = price_parent.find('span', 'a-offscreen').text
print(price)

$299.99


In [42]:
rating = item.i.text
print(rating)

4.6 out of 5 stars


In [53]:
review_count = item.find('span',{'class':'a-size-base s-underline-text'}).text
review_count = review_count.translate({ord(i): None for i in '()'})
print(review_count)

768


## generalize the pattern

In [54]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    #description and url 
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com'+atag.get('href')
    
    #price 
    try:   
        price_parent = item.find('span','a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    #rating
    try:
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base s-underline-text'}).text
        review_count = review_count.translate({ord(i): None for i in '()'})
    except AttributeError:
        rating = ''
        review_count =''
        
    result = (description , price , rating , review_count ,url)
    
    return result

In [55]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})
for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [56]:
records[0]

('LG 2023 Newest 29 Inch WFHD IPS Ultra Wide Monitor, Dual Speakers, 2560x1080, 99% sRGB, HDR10, FreeSync, 21 9, Wall Mountable, 75Hz Refresh Rate Bundle with JAWFOAL',
 '$299.99',
 '4.6 out of 5 stars',
 '40',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A083737826KZ3N881O4TS&qualifier=1676913429&id=1585949287652918&widgetName=sp_atf&url=%2FLG-inch-UltraWide-FreeSync-Monitor%2Fdp%2FB08CL1JKJ2%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dultrawide%2Bmonitor%26qid%3D1676913429%26sr%3D8-1-spons%26psc%3D1%26smid%3DA3JPM3Q1OKJJPX')

In [57]:
for row in records:
    print(row[1])

$299.99
$299.99
$259.38
$196.99
$399.99
$379.99
$196.99
$446.00
$336.99
$169.99
$329.00
$229.99
$339.99
$246.99
$296.99
$349.99
$199.99
$219.77
$199.99
$349.99
$99.99
$158.99


## getting to the next page 

In [58]:
def get_url(search_term):
    """generate a url from a search term"""
    template = 'https://www.amazon.com/s?k={}'
    search_term = search_term.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term)
    
    #add page query to url
    url += '&page{}'
    
    return url
     

## putting all together 

In [71]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver 

def get_url(search_term):
    """generate a url from a search term"""
    template = 'https://www.amazon.com/s?k={}'
    search_term = search_term.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term)
    
    #add page query to url
    url += '&page{}'
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    
    #description and url 
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com'+atag.get('href')
    
    #price 
    try:   
        price_parent = item.find('span','a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    #rating
    try:
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base s-underline-text'}).text
        review_count = review_count.translate({ord(i): None for i in '()'})
    except AttributeError:
        rating = ''
        review_count =''
        
    result = (description , price , rating , review_count ,url)
    
    return result

def main(search_term):
    """Run main program routing"""
    #startup the webdriver 
    #driver = webdriver.Firefox()
    driver = webdriver.Chrome()
    
    records = []
    url = get_url(search_term)
    
    for page in range(1 , 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source,'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
        
        #save the data to csv file
    with open('results.csv', 'w',newline ='',encoding ='utf-8') as f :
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Rating','ReviewCount' ,'URL'])
        writer.writerows(records)
            

In [72]:
main('ultrawide monitor')